In [ ]:
import pickle
import os
import pandas as pd
import numpy as np
np.random.seed(42)
from sklearn import utils
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.neighbors import KNeighborsClassifier as knc
from sklearn.model_selection import train_test_split as splt
from sklearn.metrics import matthews_corrcoef as mcc 
from sklearn.metrics import accuracy_score as acc

In [ ]:
dfx = pd.read_csv("PLIC-milano-processed.csv", sep=';')
dfy = pd.read_csv("MILANO_wSCORE.csv")

In [ ]:
# Thresholds are 0 - 0.01, 0.01 - 0.02, 0.02 - 0.05, 0.05 - 1
y = dfy["SCORE"].values/100
y_cat = np.zeros((y.shape[0]))
for i, val in enumerate(y):
    if(val < 0.01):
        y_cat[i] = 1
    elif(val < 0.02):
        y_cat[i] = 2
    elif(val < 0.05):
        y_cat[i] = 3
    else:
        y_cat[i] = 4

In [ ]:
dfx = dfx.select_dtypes(exclude=['object'])
dfx = dfx.drop(labels = ['smoking', 'smoking recod', 'glucose', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1) #
dfx = dfx.dropna(how='any', axis=1)
X = np.around(dfx.values).astype(int)

In [ ]:
X_tr, X_ts, y_tr, y_ts = splt(X, y_cat, test_size=0.25, random_state=42)
print(utils.multiclass.type_of_target(X_tr))
print(utils.multiclass.type_of_target(y_tr))

In [ ]:
from scipy import sparse
print(X_tr.shape[0]*X_tr.shape[1])
X_tr = sparse.csr_matrix(X_tr + 1)
X_ts = sparse.csr_matrix(X_ts + 1)
print(X_tr.count_nonzero())

In [ ]:
clf = rfc(n_estimators = 1000, max_depth = 100, random_state = 42)
clf.fit(X_tr, y_tr)

In [ ]:
y_ts_pred = clf.predict(X_ts)


In [ ]:
print(mcc(y_ts, y_ts_pred))

In [ ]:
print(acc(y_ts, y_ts_pred))